In [14]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

tokenizer = BertTokenizer.from_pretrained("../MARBERT_pytorch_verison/",local_files_only=True)
model = BertForMaskedLM.from_pretrained("../MARBERT_pytorch_verison/",local_files_only=True)


Some weights of the model checkpoint at ../MARBERT_pytorch_verison/ were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Test

In [16]:
from transformers import pipeline
fill = pipeline('fill-mask', model=model, tokenizer=tokenizer)

In [21]:
tokenizer.ids_to_tokens[6695]

'البرد'

In [19]:
fill("كيف نترك [MASK] بلا تخفيف")

[{'score': 0.12849706411361694,
  'token': 15071,
  'token_str': 'ا ل و ز ن',
  'sequence': 'كيف نترك الوزن بلا تخفيف'},
 {'score': 0.03307188302278519,
  'token': 4555,
  'token_str': 'ا ل ا ل م',
  'sequence': 'كيف نترك الالم بلا تخفيف'},
 {'score': 0.028317086398601532,
  'token': 17950,
  'token_str': 'ا ل ت د خ ي ن',
  'sequence': 'كيف نترك التدخين بلا تخفيف'},
 {'score': 0.024875370785593987,
  'token': 6695,
  'token_str': 'ا ل ب ر د',
  'sequence': 'كيف نترك البرد بلا تخفيف'},
 {'score': 0.019726363942027092,
  'token': 14114,
  'token_str': 'ا ل ح م ل',
  'sequence': 'كيف نترك الحمل بلا تخفيف'}]

In [23]:
# دا مجرد رأى و الله اعلم و ابقا
fill("دا مجرد رأى و[MASK] اعلم و ابقا")

[{'score': 0.8920164108276367,
  'token': 1944,
  'token_str': 'ا ل ل ه',
  'sequence': 'دا مجرد راى و الله اعلم و ابقا'},
 {'score': 0.05682622268795967,
  'token': 2410,
  'token_str': 'ر ب ن ا',
  'sequence': 'دا مجرد راى و ربنا اعلم و ابقا'},
 {'score': 0.012048384174704552,
  'token': 2043,
  'token_str': 'ا ن ا',
  'sequence': 'دا مجرد راى و انا اعلم و ابقا'},
 {'score': 0.008924596942961216,
  'token': 2188,
  'token_str': 'و ا ل ل ه',
  'sequence': 'دا مجرد راى و والله اعلم و ابقا'},
 {'score': 0.0060321856290102005,
  'token': 16954,
  'token_str': 'ر ب ى',
  'sequence': 'دا مجرد راى و ربى اعلم و ابقا'}]

In [48]:
fill("هو كل يوم نسمع فتوى جديده  ماكلنا عارفين من كان [MASK] أو على سفر وخلصت  هنخترع بقى رخص جديده من عندنا ")

[{'score': 0.3530510663986206,
  'token': 13147,
  'token_str': 'م س ا ف ر',
  'sequence': 'هو كل يوم نسمع فتوى جديده ماكلنا عارفين من كان مسافر او على سفر وخلصت هنخترع بقى رخص جديده من عندنا'},
 {'score': 0.024989185854792595,
  'token': 32403,
  'token_str': 'س ك ر ا ن',
  'sequence': 'هو كل يوم نسمع فتوى جديده ماكلنا عارفين من كان سكران او على سفر وخلصت هنخترع بقى رخص جديده من عندنا'},
 {'score': 0.02228216826915741,
  'token': 5034,
  'token_str': 'م ع ه م',
  'sequence': 'هو كل يوم نسمع فتوى جديده ماكلنا عارفين من كان معهم او على سفر وخلصت هنخترع بقى رخص جديده من عندنا'},
 {'score': 0.02149583026766777,
  'token': 2408,
  'token_str': 'ف ي ه ا',
  'sequence': 'هو كل يوم نسمع فتوى جديده ماكلنا عارفين من كان فيها او على سفر وخلصت هنخترع بقى رخص جديده من عندنا'},
 {'score': 0.019086632877588272,
  'token': 33750,
  'token_str': 'م س ج و ن',
  'sequence': 'هو كل يوم نسمع فتوى جديده ماكلنا عارفين من كان مسجون او على سفر وخلصت هنخترع بقى رخص جديده من عندنا'}]

# Get contextual word embedding 

In [1]:
import numpy as np
import torch
from transformers import  AutoModel,AutoTokenizer


def get_word_idx(sent: str, word: str):
    return sent.split(" ").index(word)


def get_hidden_states(encoded, token_ids_word, model, layers):
    """Push input IDs through model. Stack and sum `layers` (last four by default).
    Select only those subword token outputs that belong to our word of interest
    and average them."""
    with torch.no_grad():
        output = model(**encoded)

    # Get all hidden states
    states = output.hidden_states
    # Stack and sum all requested layers
    output = torch.stack([states[i] for i in layers]).sum(0).squeeze()
    # Only select the tokens that constitute the requested word
    word_tokens_output = output[token_ids_word]

    return word_tokens_output.mean(dim=0)


def get_word_vector(sent, idx, tokenizer, model, layers):
    """Get a word vector by first tokenizing the input sentence, getting all token idxs
    that make up the word of interest, and then `get_hidden_states`."""
    encoded = tokenizer.encode_plus(sent, return_tensors="pt")
    # get all token idxs that belong to the word of interest
    token_ids_word = np.where(np.array(encoded.word_ids()) == idx)

    return get_hidden_states(encoded, token_ids_word, model, layers)


c:\Users\river\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Test word similarity with cosine similarity sklearn

In [2]:
# Use last four layers by default

layers = [-4, -3, -2, -1] 
tokenizer = AutoTokenizer.from_pretrained("../MARBERT_pytorch_verison/",max_len=512)
model = AutoModel.from_pretrained("../MARBERT_pytorch_verison/", output_hidden_states=True)

Some weights of the model checkpoint at ../MARBERT_pytorch_verison/ were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
import pandas as pd
df = pd.read_csv("../Data/Preprocessed/RestOf_AOC_youm7_comments.csv")
df.head(10)

,Text
0,تحياتي لكم جميعا
1,ودي تشم ولا تقرقش
2,هو كل يوم نسمع فتوى جديده ماكلنا عارفين من كا...
3,يفطر وعدة من أيام أخر زي ما العالم ماشي من ألف...
4,بلاش اختراعات يا عم الحاج
5,الصيام صيام ادا كنت مريض فطر وعدة من ايام اخر ...
6,الدواء لابد ان يشرب في كميه مياه كافيهلوشرب ال...
7,لاسف فان تنطع بعض من يعتقدون انهم لانهم تمشيخو...
8,انها فتوى تنم عن جهل فاضح وتنطع من لا يفقه
9,اولا كان الاولى بمن قال الفتوى الجمقاء ان يقول...


In [33]:
df.iloc[2][0]

'هو كل يوم نسمع فتوى جديده  ماكلنا عارفين من كان مريضا أو على سفر وخلصت  هنخترع بقى رخص جديده من عندنا '

In [4]:
text = "هو كل يوم نسمع فتوى جديده  ماكلنا عارفين من كان مريضا أو على سفر وخلصت  هنخترع بقى رخص جديده من عندنا "
x = get_word_idx(text, "فتوى")

embd = get_word_vector(text, x, tokenizer, model, layers)


In [7]:
np.array(embd)

array([ 1.05541396e+00, -5.67821562e-01,  2.98531115e-01, -3.46632051e+00,
        1.79541743e+00, -1.21660149e+00, -3.09225023e-01, -5.62635756e+00,
        5.86506224e+00, -9.87138689e-01,  3.28087062e-02,  1.09144950e+00,
       -3.36659169e+00, -3.33140045e-01, -1.68454504e+00, -5.42061090e-01,
       -2.22262979e+00, -2.21202970e+00, -4.54788399e+00, -2.00794324e-01,
       -2.34295177e+00,  1.35216594e+00, -8.30536366e+00,  4.11758041e+00,
       -1.66416240e+00,  3.67621493e+00,  6.47384524e-01, -1.82692456e+00,
       -2.92021370e+00,  3.29810739e-01,  5.25545216e+00, -4.55368328e+00,
       -2.13490629e+00,  1.09533796e+01,  2.89962649e-01, -1.46048486e+00,
        3.59118915e+00,  9.40522909e-01,  3.73205334e-01,  2.70744300e+00,
       -2.90056038e+00,  6.66461563e+00,  2.33909631e+00,  4.12114573e+00,
       -5.27561724e-01,  2.10542846e+00,  1.75626564e+00,  2.98278284e+00,
       -2.35785890e+00, -2.01834059e+00, -7.72282743e+00, -1.00568509e+00,
        1.18760264e+00,  

In [46]:
text = "ولذلك ترى المحكمة أنها تمتلك الاختصاص في مسألة إصدار فتوى ردا على الطلب الذي تقدمت به الجمعية العامة"
x = get_word_idx(text, "فتوى")

embd_2 = get_word_vector(text, x, tokenizer, model, layers)

x2 = get_word_idx(text, "الجمعية")
embd_3 = get_word_vector(text, x2, tokenizer, model, layers)

In [47]:
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(embd_2.reshape(1,-1),embd.reshape(1,-1)))
print(cosine_similarity(embd_3.reshape(1,-1),embd.reshape(1,-1)))

[[0.8713753]]
[[0.6754375]]


In [95]:
fill("الحياه [MASK]" )

[{'score': 0.40699440240859985,
  'token': 4540,
  'token_str': 'ح ل و ه',
  'sequence': 'الحياه حلوه'},
 {'score': 0.043277643620967865,
  'token': 8244,
  'token_str': 'ع ل م ت ن ي',
  'sequence': 'الحياه علمتني'},
 {'score': 0.040454622358083725,
  'token': 5551,
  'token_str': 'ج م ي ل ه',
  'sequence': 'الحياه جميله'},
 {'score': 0.03739464655518532,
  'token': 14477,
  'token_str': 'ت ج ا ر ب',
  'sequence': 'الحياه تجارب'},
 {'score': 0.0230946633964777,
  'token': 18794,
  'token_str': 'ق ص ي ر ه',
  'sequence': 'الحياه قصيره'}]

In [109]:
%pip install torchmetrics

Note: you may need to restart the kernel to use updated packages.


In [110]:
from torchmetrics.text.bert import BERTScore

c:\Users\river\anaconda3\envs\tf\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] The specified procedure could not be found
  warn(f"Failed to load image Python extension: {e}")


In [103]:
sent = "الحياه حلوه" 
idx = get_word_idx(sent, "حلوه")

word_embedding = get_word_vector(sent, idx, tokenizer, model, layers)

sent2 = "الحياه وحشه"
idx2 = get_word_idx(sent2, "وحشه")

word_embedding2 = get_word_vector(sent2, idx2, tokenizer, model, layers)

sent3 = "الحياه جميله" 
idx3 = get_word_idx(sent3, "جميله")

word_embedding3 = get_word_vector(sent3, idx3, tokenizer, model, layers)

sent4 = "الحياه تجارب" 
idx4 = get_word_idx(sent4, "تجارب")

word_embedding4 = get_word_vector(sent4, idx4, tokenizer, model, layers)


In [104]:
sent3 = "جميله" 
idx3 = get_word_idx(sent3, "جميله")

word_embedding5 = get_word_vector(sent3, idx3, tokenizer, model, layers)

In [129]:
sentence1 ="انا رايح اركب المشروع"
sentence2 ="كان لازم اخلص المشروع فى معاده"
sentence3 ="المشروع عمل حادثه بينا فى الطريق"
id1 = get_word_idx(sentence1, "المشروع")
id2 = get_word_idx(sentence2, "المشروع")
id3 = get_word_idx(sentence3, "المشروع")

word_embedding_bus = get_word_vector(sentence1, id1, tokenizer, model, layers)
word_embedding_project = get_word_vector(sentence2, id2, tokenizer, model, layers)
word_embedding_bus_2 = get_word_vector(sentence3, id3, tokenizer, model, layers)

In [137]:
tokenizer.tokenize("انا رايح اركب المشروع")

['انا', 'رايح', 'اركب', 'المشروع']

In [147]:
print('نرم' in tokenizer.vocab.keys())
tokenizer.tokenize("انت نرم يلا")

False


['انت', 'نر', '##م', 'يلا']

In [146]:
fill("انت [MASK] يلا")

[{'score': 0.1291009485721588,
  'token': 22683,
  'token_str': 'ع ب ي ط',
  'sequence': 'انت عبيط يلا'},
 {'score': 0.08621302992105484,
  'token': 32469,
  'token_str': 'ك د ا ب',
  'sequence': 'انت كداب يلا'},
 {'score': 0.044884275645017624,
  'token': 33152,
  'token_str': 'خ و ل',
  'sequence': 'انت خول يلا'},
 {'score': 0.035633184015750885,
  'token': 14027,
  'token_str': 'ك ا ف ر',
  'sequence': 'انت كافر يلا'},
 {'score': 0.03474877402186394,
  'token': 3306,
  'token_str': 'م ي ن',
  'sequence': 'انت مين يلا'}]

In [135]:
fill("انا رايح اركب [MASK]")

[{'score': 0.07754268497228622,
  'token': 12264,
  'token_str': 'ع ج ل',
  'sequence': 'انا رايح اركب عجل'},
 {'score': 0.052763354033231735,
  'token': 33856,
  'token_str': 'ت ا ك س ي',
  'sequence': 'انا رايح اركب تاكسي'},
 {'score': 0.04821640998125076,
  'token': 30165,
  'token_str': 'ا ل ق ط ر',
  'sequence': 'انا رايح اركب القطر'},
 {'score': 0.04384848475456238,
  'token': 60969,
  'token_str': 'ت و ك ت و ك',
  'sequence': 'انا رايح اركب توكتوك'},
 {'score': 0.04264984279870987,
  'token': 1936,
  'token_str': 'u r l',
  'sequence': 'انا رايح اركب url'}]

In [136]:
fill("كان لازم اخلص [MASK] فى معاده")

[{'score': 0.07466279715299606,
  'token': 15715,
  'token_str': 'ا ل م ن ه ج',
  'sequence': 'كان لازم اخلص المنهج فى معاده'},
 {'score': 0.04484668746590614,
  'token': 11891,
  'token_str': 'ا م ت ح ا ن ا ت',
  'sequence': 'كان لازم اخلص امتحانات فى معاده'},
 {'score': 0.039108142256736755,
  'token': 22167,
  'token_str': 'ا ل م ا د ه',
  'sequence': 'كان لازم اخلص الماده فى معاده'},
 {'score': 0.030206598341464996,
  'token': 23722,
  'token_str': 'م ذ ا ك ر ة',
  'sequence': 'كان لازم اخلص مذاكرة فى معاده'},
 {'score': 0.019784992560744286,
  'token': 13301,
  'token_str': 'ا ل م ا د ة',
  'sequence': 'كان لازم اخلص المادة فى معاده'}]

## BERTScore

In [ ]:
bertscore = BERTScore(model = model,tokenizer=tokenizer)
score = bertscore(sent, sent2)

In [113]:
score

{'precision': 0.9911123514175415,
 'recall': 0.9898421168327332,
 'f1': 0.9904769062995911}

In [118]:
bertscore("دا مجرد راى و الله اعلم و ابقا", "دا مجرد راى و ربنا اعلم و ابقا")

{'precision': 0.996008038520813,
 'recall': 0.9945570230484009,
 'f1': 0.9952820539474487}

In [120]:
bertscore("دا مجرد راى و الله اعلم و ابقا", "دا مجرد فكر و الله معنا ان شاء")

{'precision': 0.991013765335083,
 'recall': 0.990456759929657,
 'f1': 0.9907351732254028}

In [121]:
bertscore("دا مجرد راى و الله اعلم و ابقا", "دا مجرد حلم و الله معنا ان شاء")

{'precision': 0.9899436831474304,
 'recall': 0.9903251528739929,
 'f1': 0.9901343584060669}

In [122]:
bertscore("دا مجرد راى و الله اعلم و ابقا", "الله معنا ان شاء")


{'precision': 0.9861608743667603,
 'recall': 0.9894135594367981,
 'f1': 0.9877845048904419}

In [125]:
bertscore(sentence1,sentence2)

{'precision': 0.9909185171127319,
 'recall': 0.9884927272796631,
 'f1': 0.9897041320800781}

## Cosine Similarity

In [106]:
word_embedding = np.array(word_embedding).reshape(1,-1)
word_embedding2 = np.array(word_embedding2).reshape(1,-1)
word_embedding3 = np.array(word_embedding3).reshape(1,-1)
word_embedding4 = np.array(word_embedding4).reshape(1,-1)
word_embedding5 = np.array(word_embedding5).reshape(1,-1)
word_embedding.shape

(1, 768)

In [107]:
cosine_similarity(word_embedding3,word_embedding5)

array([[0.88985634]], dtype=float32)

* حلوه vs وحشه

In [89]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(word_embedding,word_embedding2)

array([[0.85630375]], dtype=float32)

* حلوه vs جميله

In [90]:
cosine_similarity(word_embedding,word_embedding3)


array([[0.92183995]], dtype=float32)

* جميله vs وحشه

In [92]:
cosine_similarity(word_embedding2,word_embedding3)


array([[0.880145]], dtype=float32)

* جميله vs تجارب

In [101]:
cosine_similarity(word_embedding4,word_embedding)


array([[0.77350855]], dtype=float32)

In [133]:
cosine_similarity(word_embedding_project.reshape(1,-1), word_embedding_bus.reshape(1,-1))

array([[0.9164676]], dtype=float32)

In [130]:
cosine_similarity(word_embedding_bus.reshape(1,-1), word_embedding_bus_2.reshape(1,-1))

array([[0.87750906]], dtype=float32)

In [132]:
cosine_similarity(word_embedding_project.reshape(1,-1), word_embedding_bus_2.reshape(1,-1))

array([[0.8954044]], dtype=float32)

## Cross Entropy

In [160]:
# calculate cross entropy with keras
from numpy import asarray
from keras import backend
from keras.losses import categorical_crossentropy
# prepare classification data
p = asarray(word_embedding)
q = asarray(word_embedding2)
# convert to keras variables
y_true = backend.variable(p)
y_pred = backend.variable(q)
# calculate the average cross-entropy
mean_ce = backend.eval(categorical_crossentropy(y_true, y_pred))
print('Average Cross Entropy: %.3f nats' % mean_ce)

Average Cross Entropy: -7973.970 nats
